<a href="https://colab.research.google.com/github/liuchiente/ardiuno_and_iot/blob/main/Wiki_Green_Roach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 哈囉你好嗎

---
本程式主要是為了抓取某維基百科上資料,並轉換成資料集,資料集內容涵蓋

主題: 譬如曹錦輝
說明: 譬如史上最佳天才投手
連結: 絕對位置(不儲存錨點等相對位置)
編號: 文件編號
二階主題: 譬如牛肉麵放太多水
類型: 文字、列表、表格
關鍵字: 在文章裡出現的關鍵字、超連結
關鍵字: 在文章裡出現的關鍵字、超連結
  


#引入套件

#切換路徑到指定資料夾


In [ ]:
sys_dir = "/content/drive/My Drive/Colab Notebooks/wikigroach/dataset"
#連接並mount自己的雲端硬碟(點選跑出網址複製最後的授權碼貼上並執行)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#切換執行目錄
import os
if not os.path.isdir(sys_dir):
    os.mkdir(sys_dir)
os.chdir(sys_dir)

In [ ]:
! pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=22eefbdfcdd8c1c61d6cf36a0c5033afbcbbaa2de2b25a975d1293c1a3d123e0
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [ ]:
#引入套件
import requests
import re
import time
import json
from bs4 import BeautifulSoup
#from fake_useragent import UserAgent
import datetime
import pandas
import base64
import urllib.parse
from pathlib import Path
import numpy

In [ ]:
pure_data_path = 'pure_data.csv'
pure_data_path_drop = 'pure_data_drop.csv'
all_columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation']

In [ ]:
#建立檔案
if not os.path.isfile(pure_data_path):
  dfd=pandas.DataFrame(columns=all_columns)
  dfd.to_csv(pure_data_path_drop,index=False)

  dfn=pandas.DataFrame([(1,'http://twbsball.dils.tku.edu.tw/wiki/index.php/%E9%A6%96%E9%A0%81',2,0,'','','','')],columns=all_columns)
  dfn.to_csv(pure_data_path,index=False)


NameError: ignored

In [ ]:
  dfd=pandas.DataFrame(columns=all_columns)
  dfd.to_csv('pure_data_drop_02.csv',index=False)

#定義基本函式庫

In [ ]:
#隨機產生User angent,模擬瀏覽器行為,避免被阻擋
def set_header_user_agent():
    user_agent = UserAgent()
    return user_agent.random

#檢查是不是合法的網址
def check_url(url):
    
    #超連結類型,0未定義、1、外網、2內網、3錨點、4排除
    uri_type=0

    if url is None:
      return uri_type

    regex_ext = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'
        #r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        #r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
        
    #內網
    regex_ent = re.compile(
      #r'^(?:http|ftp)s?://' # http:// or https://
      #r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'
      #r'localhost|' #localhost...
      r'/wiki/index.php/'
      r'(\d|(%[A-F0-9]{2}))*',re.IGNORECASE)
    
    #錨點
    regex_anc = re.compile(
      #r'^(?:http|ftp)s?://' # http:// or https://
      #r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'
      #r'localhost|' #localhost...
      r'#'
      r'(\.[A-F0-9]{2})*',re.IGNORECASE)
    
    #排除
    regex_ign = re.compile(
      r'/wiki/index.php/'
      r'([^\s]+(\.(?i)(jpg|png|gif|bmp))$)|'
      r'([^\s]+(\.(?i)(css|js|ico|zip))$)|'
      r'(\S+action=edit\S+)',re.IGNORECASE)
    
    #過濾關鍵字
    py_lst = ['部落格','專欄','評論','統計信息','關於本站','使用者','加入本館 Facebook 討論社群','特殊:','編輯','請求','建置','模板:','修改','使用說明','加入本館','球員簡介整理計畫']

    #判斷值
    if re.match(regex_ext, url) is not None:
      uri_type=1
    elif re.match(regex_ign, url) is not None:
      uri_type=4
    elif re.match(regex_ent, url) is not None:
      uri_type=2
      if any(s in urllib.parse.unquote(url) for s in py_lst):
        uri_type=4
    elif re.match(regex_anc, url) is not None:
      uri_type=3
      if any(s in urllib.parse.unquote(url) for s in py_lst):
        uri_type=4


    return uri_type

#查詢資料
def req_url(link,param):
    user_agent = set_header_user_agent()
    req=requests.get(link,headers={'user-agent': user_agent,'cookie': 'over18=1'},params=param)
    #debug 網址
    rslt=BeautifulSoup(req.text,'lxml')
    return rslt

In [ ]:
#封裝資料
def pack_data(sp):
  [s.extract() for s in sp('script')]
  [s.extract() for s in sp('style')]
  #body_p=sp.body.stripped_strings
  #re_script = re.compile('<\s*script[^>]*>.*?<\s*/\s*script\s*>', re.S | re.I)
  #css_script = re.compile('<\s*style[^>]*>.*?<\s*/\s*style\s*>', re.S | re.I)
  #body_p = re_script.sub('',body_p)
  #body_p = css_script.sub('',body_p)
  body_b=base64.b64encode(sp.encode('UTF-8')).decode("UTF-8")
  return body_b

In [ ]:
#解封裝
def unpack_data(sp):
  body_d=base64.b64decode(sp).decode('UTF-8')
  rslt=BeautifulSoup(body_d,'lxml')
  return rslt

In [ ]:
df_1=pandas.read_csv("link_downloaded_01.csv")
df_2=pandas.read_csv("link_downloaded_02.csv")
df_3=pandas.read_csv("link_downloaded_03.csv")
df_4=pandas.read_csv("link_downloaded__2.csv")
df_5=pandas.read_csv("link_downloaded__3.csv")
df_6=pandas.read_csv("link_downloaded__4.csv")
df_7=pandas.read_csv("link_downloaded___2.csv")
df_8=pandas.read_csv("link_downloaded___3.csv")
df_9=pandas.read_csv("link_downloaded___4.csv")


In [ ]:
df_10=pandas.read_csv("xpure_data_d_02.csv")
print(len(df_10))

1794


In [ ]:
df_o=pandas.read_csv("pure_data_back.csv")
list_for_po=list(df_o[df_o['is_Done']==1].index.values)
df_p=df_o[df_o['is_Done']==1]
print(len(list_for_po))

6679


37228


In [ ]:
df_c=pandas.read_csv("pure_data_c-3.csv")
print(len(df_c))

43579


In [ ]:
df_all = pandas.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_p],ignore_index=True)
print(len(df_all.drop_duplicates()))
print(len(df_all))

46270
46270


In [ ]:
df_all.to_csv("link_downloaded_all.csv",index=False)

In [ ]:
for index,row in df_set.iterrows():
  no=row['No']
  idx=df_link[df_link['No']==no].index
  df_link.loc[idx,'is_Done']=1
  df_link.loc[idx,'Title']=row['Title']
  df_link.loc[idx,'KeyWords']=row['KeyWords']

list_for_p=list(df_link[df_link['is_Done']==0].index.values)
print(len(list_for_p))

26894


In [ ]:
df_link.to_csv("pure_data_c-3.csv",index=False)

In [ ]:
with open('basebale_2.json') as f1:
    data1 = json.load(f1)
with open('basebale_3.json') as f2:
    data2 = json.load(f2)
with open('basebale_4.json') as f3:
    data3 = json.load(f3)
with open('basebale_5.json') as f4:
    data4 = json.load(f4)
with open('basebale_6.json') as f5:
    data5 = json.load(f5)
with open('basebale_7.json') as f6:
    data6 = json.load(f6)
with open('basebale_8.json') as f7:
    data7 = json.load(f7)
with open('basebale_9.json') as f8:
    data8 = json.load(f8)
with open('basebale_10.json') as f9:
    data9 = json.load(f9)


In [ ]:
data_sample={}

data_sample['581']=data1['581']
data_sample['582']=data1['582']
data_sample['583']=data1['583']
data_sample['584']=data1['584']
data_sample['585']=data1['585']

In [ ]:
with open("baseball_sample.json", 'w', encoding='utf-8') as f:
    json.dump(data_sample, f, ensure_ascii=False, indent=4)

In [ ]:
print(data_sample)

{'581': {'title': '2003年第二十二屆亞洲棒球錦標賽 - 台灣棒球維基館', 'ref': ',4031:第二十二屆亞錦賽,1195:第二十二屆亞錦賽,581:第二十二屆亞錦賽,1433:第二十二屆亞錦賽,1349:第二十二屆亞錦賽,922:第二十二屆亞錦賽,3900:第二十二屆亞錦賽', 'contents': {'title': '2003年第二十二屆亞洲棒球錦標賽', 'content': [{'subject': '2003年第二十二屆亞洲棒球錦標賽', 'sno': 0, 'sections': []}, {'subject': '賽事簡介', 'sno': '1', 'sections': [{'label': 'decription', 'type': 'text', 'data': '第廿二屆亞洲棒球錦標賽由亞洲棒球總會（ＢＦＡ）主辦，於2003/10/31（五）～2003/11/7（五）在日本北海道札幌市巨蛋棒球場舉行。本屆具有參賽資格的有上屆亞錦賽前５名的國家（中華、南韓、日本、菲律賓、印尼）與2002年第五屆亞洲盃棒球賽前２名的球隊（中國、巴基斯坦）等７個國家參賽。本屆亞錦賽前兩名的球隊，可獲得2004年雅典奧運棒球賽的參賽權。最後由日本與中華順利獲得2004年的雅典奧運棒球賽參賽資格。'}]}, {'subject': '大會概要', 'sno': '2', 'sections': []}, {'subject': '各隊名單', 'sno': '3', 'sections': []}, {'subject': '中華隊', 'sno': '3.1', 'sections': [{'label': 'lists', 'type': 'list', 'data': [[{'text': '投\u3000手：張誌家(西武)、王建民(洋基2A)、許銘傑(西武)、潘威倫(統一)、杜章偉(文化)、陽建福(興農)、林岳平(台電)、林恩宇(國訓)。', 'root': 1}, {'text': '捕\u3000手：葉君璋(興農)、高志綱(合庫)。', 'root': 2}, {'text': '內野手：黃忠義(興農)、鄭兆行(興農)、張泰山(興農)、王傳家(統一)、謝佳賢(誠泰)、鄭昌明(中信)、陳鏞基(國體)。', 'root': 3

In [ ]:
data_all={}
data_all.update(data1)
data_all.update(data2)
data_all.update(data3)
data_all.update(data4)
data_all.update(data5)
data_all.update(data6)
data_all.update(data7)
data_all.update(data8)
data_all.update(data9)

34297


In [ ]:
print(data_all['582']) 

{'title': '陳金鋒 - 台灣棒球維基館', 'ref': ',6282:陳金鋒,1316:陳金鋒,1370:陳金鋒,2012:陳金鋒,1555:陳金鋒,5054:陳金鋒,1395:陳金鋒,1218:陳金鋒,1426:陳金鋒,3540:陳金鋒,4514:陳金鋒,5020:陳金鋒,3402:陳金鋒,4116:陳金鋒,2202:陳金鋒,4511:陳金鋒,5733:陳金鋒,683:陳金鋒,2151:陳金鋒,951:陳金鋒,3272:陳金鋒,1329:陳金鋒,4406:陳金鋒,3179:陳金鋒,5359:陳金鋒,1215:陳金鋒,3521:陳金鋒,2234:陳金鋒,1269:陳金鋒,2206:陳金鋒,3657:陳金鋒,5852:陳金鋒,3590:陳金鋒,5139:陳金鋒,1516:陳金鋒,1245:陳金鋒,4709:陳金鋒,1644:陳金鋒,4343:陳金鋒,594:陳金鋒,3405:陳金鋒,3332:陳金鋒,4399:陳金鋒,3029:陳金鋒,4324:陳金鋒,1743:陳金鋒,3707:陳金鋒,3397:陳金鋒,3961:陳金鋒,1624:陳金鋒,3524:陳金鋒,6487:陳金鋒,2230:陳金鋒,1261:陳金鋒,4318:陳金鋒,6003:陳金鋒,3447:陳金鋒,3838:陳金鋒,3401:陳金鋒,2582:陳金鋒,6551:陳金鋒,3373:陳金鋒,5671:陳金鋒,3379:陳金鋒,4628:陳金鋒,2315:陳金鋒,3398:陳金鋒,2457:陳金鋒,2178:陳金鋒,3624:陳金鋒,589:陳金鋒,2064:陳金鋒,4827:陳金鋒,5447:陳金鋒,4287:陳金鋒,1235:陳金鋒,5999:陳金鋒,1425:陳金鋒,3400:陳金鋒,1315:陳金鋒,4713:陳金鋒,4057:陳金鋒,4010:陳金鋒,4659:陳金鋒,3312:陳金鋒,1345:陳金鋒,2500:陳金鋒,4866:陳金鋒,3430:陳金鋒,703:陳金鋒,3344:陳金鋒,4715:陳金鋒,3618:陳金鋒,4506:陳金鋒,6170:陳金鋒,2201:陳金鋒,1820:陳金鋒,3374:陳金鋒,2058:陳金鋒,4620:陳金鋒,2289:陳金鋒,6490:陳金鋒,3345:陳金鋒,1320:陳金鋒,3403:陳金鋒,2244:陳金鋒,4508:陳金鋒,5884:陳

In [ ]:
with open("baseball_all.json", 'w', encoding='utf-8') as f:
    json.dump(data_all, f, ensure_ascii=False, indent=4)

In [ ]:
#'No','Uri','Type','is_Done','Title',Data','KeyWords','Relation'

#讀取處理資料
df_link=pandas.read_csv("pure_data_c-4.csv")
#df_link_drop=pandas.read_csv('pure_data_drop_02.csv')

#新資料
#df_link_new=pandas.DataFrame(columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation'])
#df_link_new=pandas.DataFrame(columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation'])
#df_link_url_new=pandas.DataFrame(columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation'])
#df_link_drop_new=pandas.DataFrame(columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation'])

df_link_new=pandas.DataFrame(columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation'])
df_link_url_new=pandas.DataFrame(columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation'])
df_link_drop_new=pandas.DataFrame(columns=['No','Uri','Type','is_Done','Title','Data','KeyWords','Relation'])



#目錄
#df_link_c=pandas.read_csv("pure_data_c.csv")

#一次處理 5000頁
list_for_p=list(df_link[df_link['is_Done']==0].index.values)[10000:]


#下載回所有資料
for idx in list_for_p:
  url='http://twbsball.dils.tku.edu.tw'+df_link.iloc[idx]['Uri']
  reps=req_url(url,{})

  #目前的文檔編號
  doc_id=df_link.loc[idx,'No']

  #主題 
  df_link.loc[idx,'Title']=reps.title.text
  df_link.loc[idx,'is_Done']=1
  #SEO　Keywords
  df_link.loc[idx,'KeyWords']=reps.find("meta",attrs={'name':'keywords'})['content']
  df_link.loc[idx,'Relation']=''

  #取單一筆Row
  df_link_newrow=df_link.iloc[idx].copy()
  #資料內容
  df_link_newrow['Data']=pack_data(reps)

  #另存新檔
  df_link_new=df_link_new.append(df_link_newrow,ignore_index=True)

  #整理網址
  for ae in reps.find('div',id="mw-content-text").findAll('a'):
    if ae.get('href') is not None:
      uri_type=check_url(ae.get('href'))
      uri_desc= ae.text if ae.text is not None and ae.text !='' else ae.get('title')
      #如是內網才收集
      if uri_type==2:
        indexes=df_link.index[df_link['Uri']==ae.get('href')].tolist()
        #已經存在
        if len(indexes)>0:
          r_list=str(df_link.iloc[indexes[0]]['Relation']).split(',')
          r_list.append(str(doc_id)+':'+str(uri_desc))
          r_list=list(set(r_list))
          df_link.loc[indexes[0],'Relation']=','.join(r_list)
        else:
          new_idx=df_link_url_new['No'].iloc[-1] if not df_link_url_new.empty else 0
          new_row={'No':new_idx+1,'Uri':ae.get('href'),'Type':uri_type,'is_Done':0,'Title':'','Data':'','KeyWords':'','Relation':str(doc_id)+':'+str(uri_desc)}
          df_link_url_new=df_link_url_new.append(new_row,ignore_index=True)
      else:
        indexes=df_link_drop_new.index[df_link_drop_new['Uri']==ae.get('href')].tolist()
        #已經存在
        if len(indexes)>0:
          r_list=str(df_link_drop_new.iloc[indexes[0]]['Relation']).split(',')
          r_list.append(str(doc_id)+':'+str(uri_desc))
          r_list=list(set(r_list))
          df_link_drop_new.loc[indexes[0],'Relation']=','.join(r_list)
        else:
          new_idx_d=df_link_drop_new['No'].iloc[-1] if not df_link_drop_new.empty else 0
          new_row={'No':new_idx_d+1,'Uri':ae.get('href'),'Type':uri_type,'is_Done':0,'Title':'','Data':'','KeyWords':'','Relation':str(doc_id)+':'+str(uri_desc)}
          df_link_drop_new=df_link_drop_new.append(new_row,ignore_index=True)

df_link.to_csv("xpure_data_c4.csv",index=False)
df_link_drop_new.to_csv('xpure_data_drop_03.csv',index=False)
df_link_new.to_csv("xpure_data_d_02.csv",index=False)
df_link_url_new.to_csv("xpure_data_c_new.csv",index=False)

In [ ]:
print(len(df_link_drop))

181716


In [ ]:
#主題
title=resp.find('h1',attrs = {'id' : 'firstHeading'}).text

NameError: ignored

In [ ]:
#遍歷ul
def ul_traversal(ul_nvs,uls=[]):
  for li in ul_nvs.findAll('li'):
    link=li.find('a').get('href')
    text=li.find('a').text
    asset={"uri":link,"text":text}
    uls.append(asset)
    if li.find('ul') is not None:
      ul_traversal(li.find('ul'),uls)
      

In [ ]:
#目次
uls=[]
topic=resp.find('table',attrs = {'id' : 'toc'})
if topic.find('div',attrs = {'id' : 'toctitle'}).find('h2').text=='目次':
  ul_traversal(topic.find('ul'),uls)

[{'uri': '#.E7.B0.A1.E4.BB.8B', 'text': '1 簡介'}, {'uri': '#.E5.8F.B0.E7.81.A3.E5.B7.A8.E7.A0.B2.E7.9A.84.E8.AA.95.E7.94.9F', 'text': '1.1 台灣巨砲的誕生'}, {'uri': '#.E6.97.85.E7.BE.8E.E5.85.88.E9.8B.92', 'text': '1.2 旅美先鋒'}, {'uri': '#.E5.82.B2.E8.A6.96.E5.A4.A7.E8.81.AF.E7.9B.9F', 'text': '1.3 傲視大聯盟'}, {'uri': '#.E5.9B.9E.E5.9C.8B.E8.B2.A2.E7.8D.BB', 'text': '1.4 回國貢獻'}, {'uri': '#.E7.84.A1.E5.8F.AF.E5.8F.96.E4.BB.A3.E7.9A.84.E7.AC.AC.E5.9B.9B.E6.A3.92', 'text': '1.5 無可取代的第四棒'}, {'uri': '#.E5.8F.B0.E7.81.A3.E5.B7.A8.E7.A0.B2.E7.9A.84.E8.AA.95.E7.94.9F', 'text': '1.1 台灣巨砲的誕生'}, {'uri': '#.E6.97.85.E7.BE.8E.E5.85.88.E9.8B.92', 'text': '1.2 旅美先鋒'}, {'uri': '#.E5.82.B2.E8.A6.96.E5.A4.A7.E8.81.AF.E7.9B.9F', 'text': '1.3 傲視大聯盟'}, {'uri': '#.E5.9B.9E.E5.9C.8B.E8.B2.A2.E7.8D.BB', 'text': '1.4 回國貢獻'}, {'uri': '#.E7.84.A1.E5.8F.AF.E5.8F.96.E4.BB.A3.E7.9A.84.E7.AC.AC.E5.9B.9B.E6.A3.92', 'text': '1.5 無可取代的第四棒'}, {'uri': '#.E5.9F.BA.E6.9C.AC.E8.B3.87.E6.96.99', 'text': '2 基本資料'}, {'uri': '#.E7.B6.93.E6.

In [ ]:
#使用陳金鋒當開發範例
user_agent = set_header_user_agent()
resp=req_url('http://twbsball.dils.tku.edu.tw/wiki/index.php/%E9%99%B3%E9%87%91%E9%8B%92',param={})

In [ ]:
#使用錨點遍歷所有資料
for ul in uls:
  anchor=ul['uri'].replace('#','')
  header=resp.find('span',id=anchor).text
  #contents=resp.find('span',id=anchor).parent.find_nexts()
  print(resp.find('span',id=anchor).parent)
  print(resp.find('span',id=anchor).parent.find_next_siblings())
  break


<h2><span class="editsection">[<a href="/wiki/index.php?title=%E9%99%B3%E9%87%91%E9%8B%92&amp;action=edit&amp;section=1" title="编辑段落：簡介">編輯</a>]</span> <span class="mw-headline" id=".E7.B0.A1.E4.BB.8B">簡介</span></h2>
[<h3><span class="editsection">[<a href="/wiki/index.php?title=%E9%99%B3%E9%87%91%E9%8B%92&amp;action=edit&amp;section=2" title="编辑段落：台灣巨砲的誕生">編輯</a>]</span> <span class="mw-headline" id=".E5.8F.B0.E7.81.A3.E5.B7.A8.E7.A0.B2.E7.9A.84.E8.AA.95.E7.94.9F">台灣巨砲的誕生</span></h3>, <p>　　<b>陳金鋒</b>，<a class="mw-redirect" href="/wiki/index.php/%E5%8F%B0%E5%8D%97%E7%B8%A3" title="台南縣">台南縣</a>大內鄉（現<a class="mw-redirect" href="/wiki/index.php/%E5%8F%B0%E5%8D%97%E5%B8%82" title="台南市">台南市</a>大內區）人，棒球生涯前期常用外號「<b><a class="mw-redirect" href="/wiki/index.php/%E9%8B%92%E4%BB%94" title="鋒仔">鋒仔</a></b>」後因年歲增長而升級為「<b>鋒哥</b>」。據維基百科記載，陳金鋒是台灣原住民平埔族的一支西拉雅族後裔<a class="external autonumber" href="http://zh.wikipedia.org/wiki/%E8%A5%BF%E6%8B%89%E9%9B%85%E6%97%8F" rel="nofollow">[1]</a>。而在《<a href="/wiki

In [ ]:
print(rslt_hy.find('h1',attrs = {'id' : 'firstHeading'}))
print(rslt_hy.findAll('h2')[1].find_next_siblings('h3'))
print(rslt_hy.findAll('h2')[0])

<h1 class="firstHeading" id="firstHeading" lang="zh-tw"><span dir="auto">洪一中</span></h1>
[<h3> <span class="mw-headline" id=".E5.9C.8B.E5.AE.B6.E9.9A.8A.E6.95.99.E7.B7.B4.E6.99.82.E6.9C.9F">國家隊教練時期</span></h3>, <h3> <span class="mw-headline" id=".E8.81.B7.E6.A3.92.E7.B8.BD.E6.95.99.E7.B7.B4.E6.99.82.E6.9C.9F">職棒總教練時期</span></h3>, <h3> <span class="mw-headline" id=".E5.80.8B.E4.BA.BA.E7.8D.8E.E9.A0.85.E8.88.87.E6.98.8E.E6.98.9F.E8.B3.BD.E7.B4.80.E9.8C.84">個人獎項與明星賽紀錄</span></h3>]
<h2>目次</h2>


In [ ]:
user_agent = set_header_user_agent()
req=requests.get('http://twbsball.dils.tku.edu.tw/wiki/index.php/第十四屆釜山亞運',headers={'user-agent': user_agent,'cookie': 'over18=1'},params={})
rslt=BeautifulSoup(req.text,'lxml')
print(rslt.find('h1').attr(""))

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="zh-TW">
<head>
<title>2002年第十四屆韓國釜山亞運 - 台灣棒球維基館</title>
<meta charset="utf-8"/>
<meta content="MediaWiki 1.21.0" name="generator"/>
<meta content="棒球,台灣棒球史,台灣棒球大事記,台灣棒球名人錄,台灣棒球一級棒,台灣棒球影像館,中華職棒大聯盟,職棒球員點將錄,賽事" name="keywords"/>
<link href="/wiki/index.php?title=2002%E5%B9%B4%E7%AC%AC%E5%8D%81%E5%9B%9B%E5%B1%86%E9%9F%93%E5%9C%8B%E9%87%9C%E5%B1%B1%E4%BA%9E%E9%81%8B&amp;action=edit" rel="alternate" title="編輯" type="application/x-wiki"/>
<link href="/wiki/index.php?title=2002%E5%B9%B4%E7%AC%AC%E5%8D%81%E5%9B%9B%E5%B1%86%E9%9F%93%E5%9C%8B%E9%87%9C%E5%B1%B1%E4%BA%9E%E9%81%8B&amp;action=edit" rel="edit" title="編輯"/>
<link href="/favicon.ico" rel="shortcut icon"/>
<link href="/wiki/opensearch_desc.php" rel="search" title="TwBsBall (zh-tw)" type="application/opensearchdescription+xml"/>
<link href="http://twbsball.dils.tku.edu.tw/wiki/api.php?action=rsd" rel="EditURI" type="application/rsd+xml"/>
<link href="https://creativecommons.org/lic

#設定查詢條件

In [ ]:
#基本變數
q="%5B新聞%5D"
#是否停止
to_be_contiunted=True
#頁數
page=0
#查詢結果
r_list=[]
#現在時間
today = datetime.date.today()
#開始抓文章時間,預設從上個月1號開始
#first_day = today.replace(day=1,month=today.month-1)
first_day = today - datetime.timedelta(days=1)
#預設為空
r_articles=[]
if os.path.isfile('ptt_roach.json'):
  r_json_f=open('ptt_roach.json', "r")
  r_articles=json.loads(r_json_f.read())
#最後一篇文章日期
#if len(r_articles)>0:
  #articles_last=r_articles[-1]
  #print(articles_last)
  #first_day=datetime.datetime.strptime(articles_last['d'], "%Y/%m/%d")
#結束時間,預設抓一天
#last_day = first_day + datetime.timedelta(days=1)
r_articles.clear()
#last_day = today - datetime.timedelta(days=1)
last_day = today

#查詢看板列表

In [ ]:
to_be_contiunted=True
page=0
#只抓出特定日期區間範圍的文章
while to_be_contiunted:
    page=page+1
    rq=req_url('https://www.ptt.cc/bbs/Gossiping/search',{'q':'[新聞]','page':page})
    req_articles=rq.findAll('div',attrs = {'class' : 'r-ent'})
    for req_article in req_articles:
        req_date_s=req_article.find('div',attrs = {'class' : 'date'}).text.strip().zfill(5)
        #轉出時間
        req_date=datetime.datetime.strptime(str(today.year)+"/"+req_date_s, "%Y/%m/%d")
        if req_date.date() >= first_day and req_date.date() < last_day:
          r_link=req_article.find('a').get("href")
          if req_article.find('a').text.startswith('[新聞]') and not (r_link in r_list):
            #print(req_date_s+" "+req_article.find('a').text)
            r_art=dict()
            r_art['l']=r_link
            r_art['d']=datetime.datetime.strftime(req_date,"%Y/%m/%d")
            r_list.append(r_link)
        elif req_date.date() < first_day:
          to_be_contiunted=False
          break
    time.sleep(1)
print(r_list)

['/bbs/Gossiping/M.1616169418.A.843.html', '/bbs/Gossiping/M.1616169098.A.1B1.html', '/bbs/Gossiping/M.1616167476.A.CE4.html', '/bbs/Gossiping/M.1616167288.A.7BB.html', '/bbs/Gossiping/M.1616166721.A.98A.html', '/bbs/Gossiping/M.1616166455.A.EAF.html', '/bbs/Gossiping/M.1616166185.A.31E.html', '/bbs/Gossiping/M.1616165189.A.51B.html', '/bbs/Gossiping/M.1616164941.A.637.html', '/bbs/Gossiping/M.1616164389.A.407.html', '/bbs/Gossiping/M.1616164085.A.195.html', '/bbs/Gossiping/M.1616164024.A.FA6.html', '/bbs/Gossiping/M.1616163950.A.F89.html', '/bbs/Gossiping/M.1616163518.A.EEA.html', '/bbs/Gossiping/M.1616163485.A.062.html', '/bbs/Gossiping/M.1616162361.A.AAD.html', '/bbs/Gossiping/M.1616162268.A.2AD.html', '/bbs/Gossiping/M.1616162248.A.99A.html', '/bbs/Gossiping/M.1616162098.A.337.html', '/bbs/Gossiping/M.1616161966.A.366.html', '/bbs/Gossiping/M.1616161441.A.FED.html', '/bbs/Gossiping/M.1616161261.A.7EF.html', '/bbs/Gossiping/M.1616161251.A.964.html', '/bbs/Gossiping/M.1616160914.A.6E

#儲存文章清單

In [ ]:
print(len(r_list))
print(r_list[-1])
#寫入JSON檔案
with open('ptt_roach.json', 'w', encoding='utf-8') as f:
    json.dump(r_list, f, ensure_ascii=False, indent=4)

160
/bbs/Gossiping/M.1616084624.A.C1D.html


#將文章各自爬回

In [ ]:
#上次爬文結果
r_article_contents=[]
if os.path.exists('ptt_kerker_articles.json'):
  rl_json_f=open('ptt_kerker_articles.json', "r")
  r_article_contents=json.loads(rl_json_f.read())
#文章列表清單
rc_json_f=open('ptt_roach.json', "r")
rr_list=json.loads(rc_json_f.read())

160
0


In [ ]:
#將文章清單取回
for ra_url in rr_list[0:5]:
    r_article=req_url("https://www.ptt.cc"+ra_url,{})
    r_push=r_article.findAll('div',attrs = {'class' : ['push','richcontent','article-metaline','article-metaline-right']})
    for rp in r_push:
        rp.decompose()
    r_ah=r_article.findAll('a')
    for rah in r_ah:
        rah.decompose()
    r_spa=r_article.findAll('span',attrs = {'class' : ['f2','f6','b6','b7','b3','f1','article-meta-tag','article-metaline','article-meta-value']})
    for rspa in r_spa:
        rspa.decompose()
    r_info=r_article.find('div',attrs = {'id' : 'main-content'})
    try:
      r_article_content={}
      r_article_content['id']=len(r_article_contents)+1
      r_article_content['content']=r_info.text
      r_article_contents.append(r_article_content)
      rr_list.remove(ra_url)
    except Exception as e:
      print(e)
      print(r_article)
    time.sleep(2)

#儲存收集回來的文章

In [ ]:
#剩餘未處理
print(len(rr_list))
#已經取得文章
print(len(r_article_contents))

9
151


In [ ]:
#剩餘未處理
print(len(rr_list))
#已經取得文章
print(len(r_article_contents))
#寫入JSON檔案
with open('ptt_kerker_articles.json', 'w', encoding='utf-8') as f:
    json.dump(r_article_contents, f, ensure_ascii=False, indent=4)
#寫入JSON檔案
with open('ptt_kerker.json', 'w', encoding='utf-8') as f:
    json.dump(rr_list, f, ensure_ascii=False, indent=4)

9
151


In [ ]:
rd_json_f=open('ptt_kerker_articles.json', "r")
r_article_contents=json.loads(rd_json_f.read())
r_article_contents_list=[]
for r_article_content in r_article_contents:
  r_article_content['id']=r_article_content['id']+70000
  r_article_content['content']=r_article_content['content'].replace('\n','').replace('\t','').strip()
  tidx=r_article_content['content'].find('新聞標題')
  title=''
  if  tidx>0:
    title=r_article_content['content'][tidx:101]
  else:
    title=r_article_content['content'][0:101]
  r_article_content['title']=title
  r_article_contents_list.append(r_article_content)

print(r_article_contents_list[-1])


{'id': 70151, 'content': '1.媒體來源:中央廣播電台2.記者署名:撰稿編輯：吳寧康3.完整新聞標題:EMA肯定安全有效 歐盟國家恢復接種AZ疫苗4.完整新聞內文:在歐盟藥品管理局(EMA)18日表示，阿斯特捷利康(AstraZeneca)的2019年冠狀病毒疾病(COVID-19)疫苗是「安全有效」，而且和更高的血栓風險無關後，歐洲主要國家說，他們將會恢復對這支AZ疫苗的接種。由於有AZ疫苗的接種者出現血管出血(bleeding)、血栓(blood clots)與低血小板數量(low platelet counts)等嚴重不良副作用，因此包括歐盟最大成員國的德國、法國和義大利在內，已有多國宣佈暫停接種AZ疫苗。而歐盟藥品管理局發表這項備受矚目的宣佈之際，正值世界衛生組織(WHO)和英國衛生監管當局都已表示，AZ疫苗是安全的，而且對幾個正面臨確診病例令人憂心升高的國家來說，不接種疫苗的風險反而更高。在經過旗下一個安全委員會的調查後，歐盟藥品管理局執行主任庫克(Emer Cooke)18日表示，委員會已取得一項明確的科學結論：這是一支安全且有效的疫苗，而且委員會還得到結論，這支疫苗和血栓栓塞事件或血栓的總體風險上升無關。然而，庫克也指出，他們無法「確切排除」血栓事件和疫苗之間的關聯。在歐盟藥品管理局發表聲明後，包括德國、法國、西班牙、義大利、荷蘭、葡萄牙、立陶宛、拉脫維亞、斯洛維尼亞和保加利亞也都表示，將很快恢復接種AZ疫苗。而在15日暫停使用AZ疫苗的法國指出，將從19日起恢復施打這支疫苗，法國總理卡斯特克斯(Jean Castex)將會是接種者之一。此外，法國18日也成為最新一個加強防疫限制的國家，宣佈對巴黎和其他幾個地區實施一個月的有限封城，試圖擊退已導致醫院不堪重負的第三波疫情感染。5.完整新聞連結 (或短網址):6.備註:歐洲好多國家要恢復AZ施打了不過一開始帶頭的北歐好像還在觀望?台灣也快要開始打了 你各位OK嗎--', 'title': '新聞標題:EMA肯定安全有效 歐盟國家恢復接種AZ疫苗4.完整新聞內文:在歐盟藥品管理局(EMA)18日表示，阿斯特捷利康(AstraZe'}


In [ ]:
with open('ptt_kerker_articles_final.json', 'w', encoding='utf-8') as f:
    json.dump(r_article_contents_list, f, ensure_ascii=False, indent=4)

#CKIP

In [ ]:
!pip install ckiptagger
!pip install tensorflow
!pip install gdown

#CKIP語料

In [ ]:
# -*- coding: utf-8 -*-
from ckiptagger import data_utils
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/drive/My Drive/Colab Notebooks/pttroach/data.zip
1.88GB [00:39, 47.2MB/s]


In [ ]:
# 將文章切出詞組,區分為地名、人名之類的
# -*- coding: utf-8 -*-
from ckiptagger import WS, POS, NER

rd_json_f=open('ptt_kerker_articles_final.json', "r")
r_article_contents_list=json.loads(rd_json_f.read())

r_article_contents_last=r_article_contents_list[-1]

text = r_article_contents_last['content']
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

ws_results = ws([text])
pos_results = pos(ws_results)
ner_results = ner(ws_results, pos_results)

for name in ner_results[0]:
    print(name)